In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import datetime

In [2]:
# read in the data set, extract all the ticker symbols into a list
df = pd.read_csv('./data/all_cefs.csv')
tickers = sorted(list(set(df.ticker)))
df.head()

,Unnamed: 0,ticker,Date,Open,High,Low,Close,Adj_Close,NAV,Adj_NAV,Volume
0,0,ACP,2019-03-05,11.86,11.90,11.82,11.89,11.89,12.85,12.85,54156
1,1,ACP,2019-03-04,11.91,11.95,11.84,11.88,11.88,12.84,12.84,66729
2,2,ACP,2019-03-01,11.91,11.93,11.85,11.89,11.89,12.88,12.88,80627
3,3,ACP,2019-02-28,11.81,11.87,11.79,11.87,11.87,12.89,12.89,67968
4,4,ACP,2019-02-27,11.87,11.90,11.74,11.82,11.82,12.87,12.87,123751


In [3]:
# see what data is missing
df.isna().sum()

Unnamed: 0        0
ticker            0
Date              0
Open          10850
High          10850
Low           10850
Close             0
Adj_Close         0
NAV               1
Adj_NAV           1
Volume            0
dtype: int64

In [4]:
# do some data cleaning
# if Open, High, or Low missing, set to Close
# drop row if NAV is missing
df.drop(columns=['Unnamed: 0'], inplace=True)
df.dropna(subset=(['NAV']), inplace=True)
df.Open.fillna(value=df.Close, inplace=True)
df.High.fillna(value=df.Close, inplace=True)
df.Low.fillna(value=df.Close, inplace=True)

In [5]:
# make sure nothing is missing now
df.isna().sum()

ticker       0
Date         0
Open         0
High         0
Low          0
Close        0
Adj_Close    0
NAV          0
Adj_NAV      0
Volume       0
dtype: int64

In [6]:
# set index to ticker and set date column to type datetime. check types
df1 = df.set_index(['ticker'])
df1.Date = pd.to_datetime(df1.Date)
df1.dtypes

Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj_Close           float64
NAV                 float64
Adj_NAV             float64
Volume                int64
dtype: object

In [7]:
df1.set_index('Date', append=True, inplace=True)

In [10]:
df1.sort_index(inplace=True)

# drop columns to reduce size of dataset
df1.drop(columns=['Open', 'High', 'Low'], inplace=True)

In [11]:
df1.head()

Close  Adj_Close    NAV  Adj_NAV  Volume
ticker Date                                                
ACP    2011-01-27  20.00       8.93  19.06     9.04  335449
       2011-01-28  20.10       8.98  19.05     9.03   15700
       2011-01-31  20.20       9.02  18.99     9.01    7400
       2011-02-01  20.10       8.98  19.00     9.01   10150
       2011-02-02  20.05       8.95  19.02     9.02   12050

In [12]:
# fix data errors

df1.loc[('KF','2003-01-15'), 'NAV'] = None
df1.loc[('KF','2003-01-15'), 'Adj_NAV'] = None
df1.loc[('MIN','2002-05-15'), 'NAV'] = None
df1.loc[('MIN','2002-05-15'), 'Adj_NAV'] = None

In [13]:
df1.fillna(method='ffill', inplace=True)

In [14]:
df1.to_csv('./data/all_cefs_clean.csv')

In [8]:
# read in other fund data
fdf = pd.read_csv('cef_fundem.csv')
fdf.head()

,Ticker,Category,Mkt. Cap.,Active Exp.,Net Exp.,Dist. Freq.
0,ACP,F.L,1.527958e+05,0.016755,0.023619,12.0
1,ACV,E.B,2.230959e+05,0.010566,0.022416,12.0
2,ADX,E.G,1.430408e+06,0.005337,0.005337,4.0
3,AEF,E.W.CL,6.719044e+04,0.018305,0.020418,2.0
4,AFB,M.US,3.695975e+05,0.006044,0.010214,12.0


In [9]:
fdf1 = fdf.set_index('Ticker')

In [10]:
# see what categories the funds are in
fdf.Category.unique()

array(['F.L', 'E.B', 'E.G', 'E.W.CL', 'M.US', 'E.W', 'M.CA', 'E.W.AP',
       'F.G', 'E.P', 'F.V', 'E.R', 'F.M', 'E.E', 'M.FL', 'M.NY', 'F.C',
       'M.VA', 'F.A', 'E.H', 'E.X', 'E.F', 'E.U', 'F.W', 'M.MD', 'E.W.CN',
       'E.W.EU', 'O.C', 'E.M', 'F.P', 'E.C', 'E.W.EM', 'M.NJ', 'E.W.DE',
       'E.W.AU', 'E.W.IN', 'E.W.IE', 'E.W.JP', 'E.W.KR', 'M.MA', 'M.MI',
       'M.PA', 'E.W.MX', 'M.AZ', 'M.GA', 'M.MN', 'M.NC', 'M.MO', 'M.CT',
       'M.TX', 'M.OH', 'E.W.CH', 'E.W.TW', 'M.CO', 'F.T'], dtype=object)

In [11]:
# too many categories, narrow it down to five categories below

for i in range(len(fdf1)):
    if fdf1.iloc[i,0].startswith('E.W'):
        fdf1.iloc[i,0] = 'Equity Foreign'
    elif fdf1.iloc[i,0].startswith('E.'):
        fdf1.iloc[i,0] = 'Equity US'
    elif fdf1.iloc[i,0].startswith('F'):
        fdf1.iloc[i,0] = 'Fixed Income'
    elif fdf1.iloc[i,0].startswith('M'):
        fdf1.iloc[i,0] = 'Municipal Bonds'
    else:
        fdf1.iloc[i,0] = 'Precious Metals'

In [12]:
fdf1.groupby('Category').count()

,Mkt. Cap.,Active Exp.,Net Exp.,Dist. Freq.
Category,,,,
Equity Foreign,39,39,39,39
Equity US,148,146,146,145
Fixed Income,155,154,154,155
Municipal Bonds,150,149,149,150
Precious Metals,4,4,4,0


In [13]:
fdf1.rename(columns={'Mkt. Cap.':'Mkt_Cap', 'Active Exp.':'Act_Exp', 'Net Exp.':'Net_Exp', 'Dist. Freq.':'Dist_Freq'}, inplace=True)
fdf1.head()

,Category,Mkt_Cap,Act_Exp,Net_Exp,Dist_Freq
Ticker,,,,,
ACP,Fixed Income,1.527958e+05,0.016755,0.023619,12.0
ACV,Equity US,2.230959e+05,0.010566,0.022416,12.0
ADX,Equity US,1.430408e+06,0.005337,0.005337,4.0
AEF,Equity Foreign,6.719044e+04,0.018305,0.020418,2.0
AFB,Municipal Bonds,3.695975e+05,0.006044,0.010214,12.0


In [14]:
fdf1.head()

,Category,Mkt_Cap,Act_Exp,Net_Exp,Dist_Freq
Ticker,,,,,
ACP,Fixed Income,1.527958e+05,0.016755,0.023619,12.0
ACV,Equity US,2.230959e+05,0.010566,0.022416,12.0
ADX,Equity US,1.430408e+06,0.005337,0.005337,4.0
AEF,Equity Foreign,6.719044e+04,0.018305,0.020418,2.0
AFB,Municipal Bonds,3.695975e+05,0.006044,0.010214,12.0


In [17]:
fdf1.to_csv('./data/cef_fundem_clean.csv')